In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import json
from nltk.tokenize import  word_tokenize
import tensorflow as tf
import ast

/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import word embeddings and reformat
max_sequence_length = 20

In [3]:
raw_data = pd.read_csv('/home/pratik/NER/data/large_positive.csv')
len(raw_data)

21682

In [4]:
raw_data.drop_duplicates(subset=['body'], inplace=True)
len(raw_data)

16623

In [5]:
def convert_to_list(original_text_list):
    l = ast.literal_eval(original_text_list)
    l = [i.strip() for i in l]
    return l
raw_data['original_text'] = raw_data['original_text'].apply(lambda x: convert_to_list(x))

In [6]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    for i in range(len(y)):
        y[i][0] = 1
    for original_text_words in original_text_list:
        for original_text in word_tokenize(original_text_words):
            words = word_tokenize(text)
            for i in range(len(words)):
                if words[i].lower() == original_text.lower():
                  #  print(words[i], original_text)
                    y[i][1] = 1
                    y[i][0] = 0
    return y


In [7]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    try:
        for i in range(len(y)):
            y[i][0] = 1
        for original_text_words in original_text_list:
            for original_text in word_tokenize(original_text_words):
                words = word_tokenize(text)
                for i in range(len(words)):
                    if words[i].lower() == original_text.lower():
                      #  print(words[i], original_text)
                        y[i][1] = 1
                        y[i][0] = 0
    except:
        pass
    return y


In [8]:
y_list = np.zeros([len(raw_data), max_sequence_length, 2])
for body, original_text_list, i in zip(raw_data['body'], raw_data['original_text'], range(len(raw_data))):
    
    y_list[i] = generate_y(body, original_text_list, i)

In [9]:
word_tokenize(raw_data['body'].iloc[11]), raw_data['original_text'].iloc[11], y_list[11]

(['No', 'I', 'ask', 'Paytm', 'debit', 'card', 'queries'],
 ['paytm debit'],
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [10]:
char_vocab = []
def get_character_set(text):
    for char in text:
        if not char in char_vocab:
            char_vocab.append(char)

raw_data['body'].apply(lambda x: get_character_set(x))

0        None
1        None
2        None
3        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
         ... 
21287    None
21288    None
21289    None
21290    None
21291    None
21292    None
21293    None
21294    None
21295    None
21296    None
21297    None
21298    None
21299    None
21301    None
21302    None
21304    None
21305    None
21306    None
21307    None
21309    None
21310    None
21311    None
21312    None
21313    None
21316    None
21318    None
21319    None
21320    None
21321    None
21322    None
Name: body, Length: 16623, dtype: object

In [11]:
char_vocab_len = len(char_vocab)

In [12]:
char_one_hot = np.zeros([char_vocab_len, char_vocab_len])
for i in range(len(char_vocab)):
    char_one_hot[i][i] = 1
    

In [13]:
max_char_length = 16
max_sequence_length = 20
char_dimensions = char_vocab_len
batch_size = 100
raw_data_len = len(raw_data)
ids_char = np.zeros([(raw_data_len),max_sequence_length,max_char_length ])
ids_char = ids_char + len(char_vocab) -1

In [14]:
g = 0
for i in (range((raw_data_len))):
    body = raw_data.iloc[i]['body']
    words = word_tokenize(body)
    for j in (range(len(words))):
        word = words[j]
        for k in range(len(word)):
            char = word[k]
            try:
                ids_char[i][j][k] = char_vocab.index(char)
            except:
                pass

In [15]:
ids_char.shape

(16623, 20, 16)

In [16]:
import tensorflow as tf
tf.reset_default_graph()


In [17]:
#weights = {'W_conv1': tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_conv2':                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_fc':tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }



#biases = {'b_conv1':tf.get_variable("b_conv1", shape=([32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_conv2':tf.get_variable("b_conv2", shape=([64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_fc':tf.get_variable("b_fc", shape=([1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }
weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
           'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
           'W_fc':tf.Variable(tf.random_normal([(int(max_char_length//2))*(int(char_dimensions//2))*32,100])),
           'out':tf.Variable(tf.random_normal([100, 10]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
           'b_conv2':tf.Variable(tf.random_normal([64])),
           'b_fc':tf.Variable(tf.random_normal([100])),
           'out':tf.Variable(tf.random_normal([10]))}

def character_level_embedding(char_input_data):
    print('sdiugfh', char_input_data)
    char_input_data = tf.cast(char_input_data, dtype=tf.int32)
    char_one_hot_tensor = tf.convert_to_tensor(char_one_hot, dtype=tf.int32)
    char_data = tf.placeholder(shape = (max_sequence_length,max_char_length), dtype=tf.int32)
    char_data = tf.nn.embedding_lookup(char_one_hot_tensor,char_input_data)
    print('char_data', char_data)
    keep_rate = 1
    print('aya')
    n_nodes_hl1 = 100
    #print(char_data.shape)
#The place holder to hold the values of new x and y at different times.
    x = tf.placeholder(dtype=tf.float32, shape=[max_sequence_length, max_char_length * char_dimensions])
    x = tf.cast(tf.reshape(char_data, shape=[max_sequence_length, max_char_length * char_dimensions]), dtype=tf.float32)
    #print(x)
#y = tf.placeholder('float', [batch_size, n_classes])

    def embedding_layer(data):
        print('aya')
        hidden_1_layer = {'weights':tf.Variable(tf.random_normal([max_char_length * char_dimensions, n_nodes_hl1])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
        l1 = tf.add(tf.tensordot(data,hidden_1_layer['weights'], axes=1), hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)
        print(l1.shape)
        return l1

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

    def maxpool2d(x):
        #                        size of window         movement of window
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


    def convolutional_neural_network(x):
        
#        tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#        
#                        tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())

        
        

        print('og', x.shape)
        x = tf.reshape(x, shape=[max_sequence_length,max_char_length,char_dimensions, 1])

        conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
        conv2 = maxpool2d(conv1)

        #conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
        #conv2 = maxpool2d(conv2)
        
        print('con2',conv2.shape)
        fc = tf.reshape(conv2,[max_sequence_length, (max_char_length//2) * (char_dimensions//2) * 32])
        fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
        fc = tf.nn.dropout(fc, keep_rate)
        #print(fc.shape)
        return fc

    def train_neural_network(x):    
        #embedded_x = embedding_layer(x)
        #embedded_x = tf.reshape(x, [max_sequence_length, char_dimensions])
        convoluted_x = convolutional_neural_network(x)
        
        return convoluted_x
    
    char_data = train_neural_network(x)
    return char_data


char_input_data = tf.placeholder(dtype=tf.float32,shape= [batch_size, max_sequence_length, max_char_length])
char_embeddings = tf.map_fn(lambda x: character_level_embedding(x), char_input_data, dtype=tf.float32)



sdiugfh Tensor("map/while/TensorArrayReadV3:0", shape=(20, 16), dtype=float32)
char_data Tensor("map/while/embedding_lookup:0", shape=(20, 16, 158), dtype=int32)
aya
og (20, 2528)
con2 (20, 8, 79, 32)


In [18]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    p = ids_char.astype(int)[100:200]
    print(type(p[0][0][0]))
    b = sess.run([char_embeddings], feed_dict={char_input_data: p})
    print(b[0].shape)
    

<class 'numpy.int64'>
(100, 20, 100)


In [19]:
lstmUnits = 64
numClasses = 2
epochs = 10
numDimensions = 100
batchSize = 100
train_len = 14000


y_train = y_list[:train_len]
y_test = y_list[train_len:]
ids_char_train = ids_char[:train_len]
ids_char_test = ids_char[train_len:]




#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test, ids_char_train, ids_char_test = train_test_split(ids,y_list,ids_char, test_size=.2)

len(y_test), len(ids_char_train)

(2623, 14000)

In [20]:
def getTrainBatch(j):
    i = j*batchSize
    labels = y_train[i:i+batchSize]
    arr_char = ids_char_train[i:i+batchSize]
    #print(labels[0])
    return labels, arr_char

In [21]:
def getTestBatch(j):
    i = j*batchSize
    labels =   y_test[i:i+batchSize]
    arr_char = ids_char_test[i:i+batchSize]
    return labels, arr_char

In [22]:
labels = tf.placeholder(tf.float32, [batchSize, max_sequence_length, numClasses])

In [23]:

print(char_embeddings.shape)
data2 = tf.placeholder(shape = (batchSize,max_sequence_length,char_dimensions), dtype=tf.float32)
data2 = char_embeddings

(100, 20, 100)


In [24]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [25]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data2,dtype=tf.float32)

In [26]:
#print(outputs)

outputs = tf.concat(outputs, 2)
#print(outputs.shape)

In [27]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,max_sequence_length,numClasses]))

In [28]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
prediction_arg = tf.argmax(prediction,axis=2)
label_arg = tf.argmax(labels,axis=2)
#false_positives
false_positives = tf.metrics.false_positives(predictions=prediction_arg, labels=label_arg)
false_negatives = tf.metrics.false_negatives(predictions=prediction_arg, labels=label_arg)
true_positives = tf.metrics.true_positives(predictions=prediction_arg, labels=label_arg)
true_negatives = tf.metrics.true_negatives(predictions=prediction_arg, labels=label_arg)
prec = tf.metrics.precision(predictions=prediction_arg, labels=label_arg)
rec = tf.metrics.recall(predictions=prediction_arg, labels=label_arg)
#false_positives = 1
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    #tf.initialize_all_variables().run()
    sess.run(init_g)
    sess.run(init_l)
    nextBatchLabels, nextChars = getTrainBatch(2)
        #print(j/2)
    pred, pa, la, fp, fn, tp, tn = sess.run([prediction, prediction_arg, label_arg, false_positives, false_negatives, true_positives, true_negatives], {labels: nextBatchLabels, char_input_data: nextChars})
    print(pa.shape, la.shape, fp, fn, tp, tn)
    #a = pred[0]
    #b = nextBatchLabels
   # print(a.shape,b.shape)
    #print(a.shape)
    #print(np.argmax(a, axis=2)[0])
    #print(np.argmax(b, axis=2)[0])
    #print(np.logical_and(np.argmax(a, axis=2)[0], np.argmax(b, axis=2)[0] ))
    #print((np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)))[0])
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=1))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=0))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2))))
correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


(100, 20) (100, 20) (1241.0, 1241.0) (122.0, 122.0) (35.0, 35.0) (602.0, 602.0)


In [29]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [30]:
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())#

#    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
#    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

In [31]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [32]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)

for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    losses = 0
    for j in range(len(y_train)//batchSize):
        nextBatchLabels, nextChars = getTrainBatch(j);
        #print(j/2)
        ll, _,pred,lab = sess.run([loss, optimizer,prediction,labels], {labels: nextBatchLabels, char_input_data: nextChars})
        print(ll)
        losses += ll

    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, {labels: nextBatchLabels, char_input_data: nextChars})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    print(losses)
writer.close()

epoch:  0
1.7768425
0.3220034
0.3342517
0.30093792
0.24523199
0.22072306
0.39846656
0.19752356
0.21552847
0.19921467
0.19564985
0.22423655
0.18145846
0.16780351
0.1640233
0.17545664
0.17453387
0.15275241
0.15850894
0.1288309
0.15634806
0.12942027
0.15531006
0.13778074
0.13107188
0.16004756
0.14203192
0.13051395
0.13081013
0.12706344
0.12769744
0.12293118
0.14803386
0.14282013
0.14243232
0.13437441
0.124914855
0.12813114
0.10810647
0.12000381
0.13182196
0.101841584
0.1162445
0.12480768
0.12589204
0.118506074
0.10866873
0.10723721
0.10456845
0.1035269
0.14309695
saved to models/pretrained_lstm.ckpt-0
0.13269171
0.14584315
0.11950712
0.13843185
0.12489086
0.1348496
0.10931877
0.13420919
0.12224957
0.14860682
0.1290848
0.12796812
0.14009927
0.13037829
0.12811087
0.12400167
0.11519429
0.11208197
0.13825457
0.12482426
0.12537818
0.11875339
0.11546525
0.1018091
0.098726444
0.10588621
0.107112885
0.0925314
0.08988948
0.08730939
0.10798127
0.16937879
0.09542506
0.09458693
0.090146974
0.0973519


0.057029616
0.0665728
0.08479553
0.109960385
0.105542585
0.10958178
0.108622335
0.08135376
0.11037512
0.09425148
0.113204576
0.07531386
0.09195281
10.49299469590187
epoch:  5
0.10215947
0.09106584
0.08192774
0.07404542
0.06076693
0.08597425
0.072587036
0.07722122
0.0935208
0.06649339
0.061067786
0.08080203
0.07617588
0.083504364
0.083558045
0.10306912
0.08635414
0.07733881
0.07111
0.06033246
0.07774826
0.062468257
0.07159346
0.06301481
0.061762325
0.0633631
0.06854009
0.048946284
0.08646109
0.06124296
0.07671655
0.08825052
0.110673696
0.11244344
0.10764082
0.10770507
0.08507918
0.09545625
0.087418824
0.080971375
0.08282491
0.074863195
0.08436833
0.08170999
0.08289487
0.10326352
0.10674257
0.110597424
0.078012444
0.11776166
0.12347265
saved to models/pretrained_lstm.ckpt-5
0.11248079
0.14175098
0.09520532
0.22198178
0.17025359
0.14258154
0.132881
0.17847757
0.12077946
0.15271622
0.13108172
0.1730445
0.11916957
0.10739105
0.1023301
0.09071233
0.083703354
0.1187327
0.13848731
0.10585587
0

0.07553086
0.067568295
0.093341194
0.07681617
0.0693346
0.08774212
0.077123396
0.099493854
0.11499844
0.10506601
0.097129576
0.098790385
0.07286002
0.08651948
0.06694964
0.07077445
0.07080132
0.08659364
0.11712522
0.11107909
0.105694
0.12868325
0.10862299
0.09783691
0.082686305
0.060455598
0.07096757
0.08664555
12.62854128330946


In [33]:
test_batches = 25
fin_pred = np.zeros([test_batches*100, 20, 2])
jj = 0
for j in range(test_batches):
    jj = j * 100
    nextBatchLabels, nextChars = getTestBatch(j);
    pred, acc, tp, tn, fp, fn, p, r = sess.run([prediction, accuracy, true_positives, true_negatives, false_positives, false_negatives, prec, rec], {labels: nextBatchLabels, char_input_data: nextChars})
    print(pred.shape)
    fin_pred[jj:jj+100] = pred

print(tp,tn,fp,fn,p,r)
    #    for i in range(10):

#        print(pred[i][:6])
#        print(nextBatchLabels[i][:6])
#        print('----------')

(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(3926.0, 3926.0) (43637.0, 43637.0) (1900.0, 1900.0) (537.0, 537.0) (0.678076, 0.67387575) (0.8790285, 0.87967736)


In [34]:
prediction_analysis = {'original_text': [], 'pred_original_text': [], 'body': []}
#for i in range(len(y_test)):
for i in range((test_batches*100)):
    body = word_tokenize(str(raw_data[train_len:].iloc[i]['body']))
    lab_index_test = np.where(np.argmax(y_test, axis=2)[i]==1)[0]
    lab_index_pred = np.where(np.argmax(fin_pred, axis=2)[i]==1)[0]
    original_text = []
    pred_original_text = []
    for j in lab_index_test:
        original_text.append(body[j])
    for j in lab_index_pred:
        pred_original_text.append(body[j])

    prediction_analysis['original_text'].append(original_text)
    prediction_analysis['pred_original_text'].append(pred_original_text)
    prediction_analysis['body'].append(body)

In [35]:
prediction_analysis_pandas = pd.DataFrame.from_dict(prediction_analysis)

In [36]:
tru_pos = 0
fals_pos = 0 
fals_neg = 0
true_pos = []
false_pos = []
false_neg = []
def check_tp(original_text, pred_original_text):
    global tru_pos, fals_pos, fals_neg
    #print((set(original_text)))
    #print((set(pred_original_text)))
    og = set(original_text)
    pr = set(pred_original_text)
    res = og.intersection(pr)
    true_pos.append(res)
    
    false_pos.append(pr - res)
    false_neg.append(og - res)
    tru_pos += len((set(original_text).intersection(pred_original_text)))
    fals_pos += len(pr -res)
    fals_neg += len(og - res)
    
for i in range(2500):
    check_tp(prediction_analysis_pandas.iloc[i]['original_text'], prediction_analysis_pandas.iloc[i]['pred_original_text'])

In [37]:
pp = tru_pos/ (tru_pos + fals_pos)
rr = tru_pos/ (tru_pos + fals_neg)
tru_pos, fals_pos, fals_neg, pp, rr, (2*pp*rr)/(pp+rr)

(3919, 1892, 517, 0.6744106005850973, 0.883453561767358, 0.7649068019908265)

In [38]:
prediction_analysis_pandas['false_positives'] = false_pos
prediction_analysis_pandas['true_positives'] = true_pos
prediction_analysis_pandas['false_negatives'] = false_neg

In [39]:
prediction_analysis_pandas

,body,original_text,pred_original_text,false_positives,true_positives,false_negatives
0,"[i, call, on, just, dial, .., its, better, the...",[samsung],"[call, just, dial]","{call, dial, just}",{},{samsung}
1,"[i, want, ambala, bus, stand, context, no, .]","[bus, stand]","[want, ambala, bus, stand]","{want, ambala}","{stand, bus}",{}
2,"[Beauty, parlour, for, women]","[Beauty, parlour]","[Beauty, parlour, women]",{women},"{Beauty, parlour}",{}
3,"[Its, not, a, night, club, dear]","[night, club]","[Its, night, club]",{Its},"{night, club}",{}
4,"[More, Nearby, :, workshop]",[workshop],[workshop],{},{workshop},{}
5,"[Not, send, me, agai]",[agai],"[Not, send]","{Not, send}",{},{agai}
6,"[Car, workshop]","[Car, workshop]",[workshop],{},{workshop},{Car}
7,[Daraganj],[Daraganj],[Daraganj],{},{Daraganj},{}
8,"[Look, for, him]","[for, him]","[Look, for, him]",{Look},"{for, him}",{}
9,"[Nearby, sbi, branch]","[sbi, branch]","[sbi, branch]",{},"{sbi, branch}",{}
